# Colab Preparations

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd 'gdrive/My Drive/Colab Notebooks/MSC-RecSys-Project'
!pip install optuna
!pip install pytorch-ignite

# Imports

In [ ]:
import torch
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from src.hyperparams_tuning import tune_params, final_train

In [ ]:
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# MovieLens 1M

In [ ]:
max_epochs_ml = 10

## HyperParameter Tuning

In [ ]:
n_trials = 2

study_ml, df_tuning_results_ml = tune_params(
    model_name='AutoRec', 
    dataset_name='movielens', 
    max_epochs=max_epochs_ml,
    n_trials=n_trials,
    device=device
    )

In [ ]:
display(df_tuning_results_ml.sort_values(by='value').head(15))
best_params_ml = study_ml.best_params

## Full Training
- Use the best hyperparameters to build the final model
- Train the final model on the train+validation data sets (full_train)
- Test it against the test set for final results

In [ ]:
test_loss_ml, final_model_ml, test_mrr_ml = final_train(
    model_name='AutoRec', 
    dataset_name='movielens', 
    best_params=best_params_ml,
    max_epochs=max_epochs_ml, 
    device=device
    )
print(f'Final test loss = {test_loss_ml}\nFinal test MRR = {test_mrr_ml}')

# Books

In [ ]:
max_epochs_bk = 1
mrr_threshold_bk = 8

## HyperParameter Tuning

In [ ]:
n_trials = 1

study_bk, df_tuning_results_bk = tune_params(
    model_name='AutoRec', 
    dataset_name='books', 
    max_epochs=max_epochs_bk,
    n_trials=n_trials, 
    device=device,
    mrr_threshold=mrr_threshold_bk
    )

In [ ]:
best_params_bk = study_bk.best_params
df_tuning_results_bk.sort_values(by='value').head(15)

## Full Training
- Use the best hyperparameters to build the final model
- Train the final model on the train+validation data sets (full_train)
- Test it against the test set for final results

In [ ]:
test_loss_bk, final_model_bk, test_mrr_bk = final_train(
    model_name='AutoRec', 
    dataset_name='books', 
    best_params=best_params_bk,
    max_epochs=max_epochs_bk, 
    device=device,
    mrr_threshold=mrr_threshold_bk
    )
print(f'Final test loss = {test_loss_bk}\nFinal test MRR = {test_mrr_bk}')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=.